# Getting financial statement footnote text from XBRL data

This notebook contains example Python code to extract text of footnotes to the financial statements as reported in XBRL 10-K using [Financial Statements and Notes Data Sets](https://www.sec.gov/dera/data/financial-statement-and-notes-data-set.html) provided by the Office of Structured Disclosure at the SEC.

**Made by:**  [Roman Chychyla](https://people.miami.edu/profile/rxc303@miami.edu)

## Textual information in XBRL corporate reports - Background

### What textual information is included in XBRL?
XBRL filings include text blocks of financial statement footnotes *only*, and not of other sections (e.g., business description, risk factors, or management's discussion and analysis sections). Nevertheless, extracting individual footnote textual information from regular plain-text/HTML filings is significantly more difficult compared to text in other sections (because financial statement footnotes greatly vary in formatting and placement across filers). For Python examples on how to extract text of sections other than financial statement footnotes, see [Using Python for Text Analysis in Accounting Research](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3576098).

### What do I need to know to extract a specific footnote from XBRL 10-K document?
The most important piece of information to know is a set of possible XBRL tags used to represent the text blocks of a specific footnote. For example, Significant Accounting Policies footnote can be represented using one of the following standard US-GAAP XBRL tags:

- *SignificantAccountingPoliciesTextBlock*
- *BasisOfPresentationAndSignificantAccountingPoliciesTextBlock*
- *OrganizationConsolidationAndPresentationOfFinancialStatementsDisclosureAndSignificantAccountingPoliciesTextBlock*

On the other hand, income tax footnote text is tagged using only one XBRL concept:
- *IncomeTaxDisclosureTextBlock*

### How do I identify XBRL tags for a given footnote?
You can search for a specific tags used to represent footnotes either through [XBRL Taxonomy Viewer](https://xbrlview.fasb.org) viewer or by downloading taxonomy files from the [FASB's website](https://fasb.org/Page/PageContent?PageId=/xbrl/2022financial.html).

Another option is to examine a sample of XBRL 10-K filings to identify common XBRL tags used for footnotes:

[Microsoft, 2021 10-K](https://www.sec.gov/ix?doc=/Archives/edgar/data/789019/000156459021039151/msft-10k_20210630.htm)

[Home Depot, 2020 10-K](https://www.sec.gov/ix?doc=/Archives/edgar/data/0000354950/000035495021000089/hd-20210131.htm)

[General Motors, 2020 10-K](https://www.sec.gov/ix?doc=/Archives/edgar/data/1467858/000146785822000034/gm-20211231.htm)

## Extracting textual information from XBRL reports: practical considerations

While it is possible to identify, collect, and parse text blocks from raw XBRL filings (as submitted to the SEC by the filers), it is a difficult task. Therefore, the Office of Structured Disclosure (OSD) created publicly available datasets with numeric and textual data extracted from XBRL filings. The benefit of these datasets is that they are reported in a tabular format and textual data has been converted from HTML format to plain-text.

The datasets are available through this [link](https://www.sec.gov/dera/data/financial-statement-and-notes-data-set.html) and are organized into year-quarter (before 2020) and year-month (since 2020) archive files. Each such archive contains information about the XBRL filings filed in the given *calendar period* (e.g., accession number, company name, report date, etc.) and XBRL data.

Therefore, we can use Python (or other programming language) to:
 1) read filing information table,
 2) read tables with data and keep only relevant footnotes (e.g., income tax footnote), and
 3) merge filing information and textual data into one table.

The output dataset can be further used for automated textual analysis.

## Example: Getting Significant Accounting Policies footnote from XBRL reports using Python

To illustrate how to extract footnote texts from multiple XBRL reports and put them in a table, we will focus on Significant Accounting Policies footnote and use one month of XBRL data.

In [ ]:
# python library to work with files and folders paths (locations)
from pathlib import Path

# we will set the current folder (Path.cwd()) as a working directory
working_folder = Path.cwd()

# set month and year input parameters (these can be modified, if needed)
year = 2021
month = 1

# we will consider 10-K and 10-K/A reports. 
# Sometimes companies are late to file XBRL data in their 10-Ks, and include XBRL data in subsequent amendments (10-K/As)
form_types = ['10-K','10-K/A']

# specify a list of XBRL footnote tags to be extracted
footnote_tags = ['SignificantAccountingPoliciesTextBlock','BasisOfPresentationAndSignificantAccountingPoliciesTextBlock','OrganizationConsolidationAndPresentationOfFinancialStatementsDisclosureAndSignificantAccountingPoliciesTextBlock']

First, we need to download the archive file with XBRL data for the desired period. This can be done either manually or programmatically. We will choose the later because such code can be reused in the future.

In [ ]:
# library to handle HTTP requests and downloads
import requests
# library for file operations
import shutil
# library for working with (zip) archives
import zipfile

# define a function to download OSD's XBRL parsed data for a given year/month (only works for years after 2020)
# this needs to be modified for years before 2020, because that data is at the quarterly data;
def download_xbrl_fin_notes(year,month):
    # generate the archive file name based on the input year and month
    filename =  f'{year}_{month:02}_notes.zip'
    # generate the download link
    url = f'https://www.sec.gov/files/dera/data/financial-statement-and-notes-data-sets/' + filename
    # set the output file path
    output_file = working_folder / filename
    # do nothing if the output file already exists
    if not output_file.exists():
        # else
        # create a HTTP request to download the file
        req = requests.get(url, stream = True)
        # if the request is successful (with status code of 200), proceed
        if req.status_code == 200:
            # create a file object to write contents of the archive
            with open (working_folder / filename,'wb') as file:
                # download the file
                shutil.copyfileobj(req.raw,file)
            # once downloaded, extract the archive to a folder of the same name
            with zipfile.ZipFile(output_file,'r') as zip:
                # create the output folder
                output_folder = working_folder / output_file.stem
                output_folder.mkdir(exist_ok=True,parents=True)
                # extract data in the output folder
                zip.extractall(output_folder)

In [ ]:
# apply the above function to the given year month
download_xbrl_fin_notes(year,month)

Now, we are ready to read and process XBRL data. We will focus on two files inside the folder with extracted archive contents:

1) **sub.tsv** - contains information about XBRL filings and their filers;
2) **txt.tsv** - contains textual content extracted from XBRL filings.

The extension *.tsv* stands for "tab-separated values". It is similar to *.csv* (comma-separated values) format, but uses tab characters instead of commas (useful, when there are comma in values such as company names).

In [ ]:
# library to work with tabular data
import pandas as pd

# generate file path of the sub.tsv file
sub_file = working_folder / f'{year}_{month:02}_notes' / 'sub.tsv'

# read the file into pandas dataframe (table)
df_sub = pd.read_csv(sub_file, sep = '\t')

# display the first five records of the dataframe to check that the data loaded correctly and get a sense of its structure
df_sub.head()

We will focus on the key variables in this dataset (and ignore the rest):
- `adsh` - Accession Number. The 20-character string formed from the 18-digit number assigned by the Commission to each EDGAR submission;
- `cik` - Central Index Key (CIK). Ten digit number assigned by the Commission to each registrant that submits filings;
- `name` - Name of registrant. This corresponds to the name of the legal entity as recorded in EDGAR as of the filing date;
- `sic` - Standard Industrial Classification (SIC). Four digit code assigned by the Commission as of the filing date, indicating the registrant's type of business.
- `form` - The submission type of the registrant's filing;
- `period` - Date of Balance Sheet, rounded to nearest month-end;
- `filed` - The date of the registrant's filing with the Commission.

In [ ]:
# keep only to the variables of interest
df_sub = df_sub[['adsh','cik','name','sic','form','period','filed']]

# display the first rows of the resulting dataframe
df_sub.head()

In addition, we will only consider 10-Ks and 10-K/A filings.

In [ ]:
# keep only obserations that correspond to 10-K and 10-K/A form types
df_sub_filtered = df_sub[df_sub['form'].isin(form_types)]

# print the first 10 records of the new dataframe
df_sub_filtered.head(10)

Next, we will use Python to read the dataset with XBRL text content, **txt.tsv**.

In [ ]:
# generate file path to the txt.tsv file
txt_file = working_folder / f'{year}_{month:02}_notes' / 'txt.tsv'

# read the file into df_txt pandas dataframe
df_txt = pd.read_csv(txt_file, sep = '\t')
# display the first five rows
df_txt.head()

To make the table simpler, we will only consider the following variables:
- `adsh` - Accession Number. The 20-character string formed from the 18-digit number assigned by the Commission to each EDGAR submission;
- `tag` - XBRL tag (concepts) of a given observation;
- `qtrs` - The count of the number of quarters represented by the data value, rounded to the nearest whole number. Value 0 represent point-in-time values;
- `ddate` - The end date for the data value, rounded to the nearest month end;
- `value` - The actual value of the concept (in our case, text);

In [ ]:
# keep only to the variables of interest
df_txt = df_txt[['adsh','tag','qtrs','ddate','value']]

# display the first rows of the resulting dataframe
df_txt.head()

Next we will merge the submission dataframe (the one we create before) with the current (textual data) dataframe. This will a) ensure that we consider only 10-K and 10-K/A filings (that we filtered on previously) and b) add submission information to the textual data. The primary key (unique observation identifier) in the both data sets is the same -  **adsh** (accession number). We will merge the two datasets based on the values of **adsh**.

In [ ]:
# merge the datasets
df_txt_filtered = df_sub_filtered.merge(df_txt, on = 'adsh')
# display the first five records
df_txt_filtered.head()

Finally, we only need to keep text of Significant Accounting Policy footnote. That is, we will only keep observations with tag names matching names in the previously-defined `footnote_tags` list.

In [ ]:
# keep only records corresponding to Significant Accounting Policies
df_fn = df_txt_filtered[df_txt_filtered['tag'].isin(footnote_tags)].reset_index(drop=True)
# display the first ten records
df_fn.head(10)

Finally, we will save the resulting table as a tab-separated values file.

In [ ]:
# generate the output file path
footnote_output_path = working_folder / (f'{year}_{month:02}_footnote_text.tsv')

# save the final dataframe to the file
df_fn.to_csv(footnote_output_path, sep="\t", index=False)

## Bonus: What to do next?

After we extract footnotes we can apply textual analysis methods to the extracted text.

For example, let's take a random footnote from the dataset:

In [ ]:
# get footnote text for the third observation
# in Python numbering starts with 0; therefore third observation has index (sequantial number) of 2
text = df_fn.iloc[0]['value']

print(text)

We can apply textual analysis methods to this text. For instance, let's extract all the words in the footnote: 

In [ ]:
# library to work with text patters using regular expression language
import re
# regular expression '\b[a-zA-Z\'\-]+\b' searches for all words in text, allowing apostrophes and hyphens in words, e.g., company's, state-of-the-art
rx_word = re.compile(r'\b[a-zA-Z\'-]+\b')

#extract all words
words = rx_word.findall((text))
# output the first 100 words
words[:100]

Now we can count the length of footnote as the total number of words:

In [ ]:
print('Number of words in the footnote:',len(words))

We can automate this procedure and calculate lengths of all footnotes in our final dataset:

In [ ]:
# library to work with text patters using regular expression language
import re
# regular expression '\b[a-zA-Z\'\-]+\b' searches for all words in text, allowing apostrophes and hyphens in words, e.g., company's, state-of-the-art
rx_word = re.compile(r'\b[a-zA-Z\'-]+\b')

def count_words(text):
    """Counts number of words in the input text."""
    
    #extract all words
    words = rx_word.findall((text))
    # output the word count
    return len(words)

In [ ]:
# create a new variable fn_length by applyting count_words to the text of each footnote in the table
df_fn['fn_length'] = df_fn.apply(lambda row: count_words(row['value']), axis = 1)

# display the first ten records of the dataset
df_fn.head(10)